In [1]:
#Import dependencies.
import pandas as pd

In [5]:
#Import files.
file1 = "salary_data_cleaned.csv"
file2 = "advisorsmith_cost_of_living_index.csv"

df = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df.head()

,Unnamed: 0,Job_Title,Salary_Estimate,Rating,Company_Name,Location,Size,Founded,Type_of_ownership,Sector,Revenue
0,0,Data Scientist,$53K-$91K (Glassdoor est.),3.8,Tecolote Research\n3.8,"Albuquerque, NM",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,$50 to $100 million (USD)
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),3.4,University of Maryland Medical System\n3.4,"Linthicum, MD",10000+ employees,1984,Other Organization,Health Care,$2 to $5 billion (USD)
2,2,Data Scientist,$80K-$90K (Glassdoor est.),4.8,KnowBe4\n4.8,"Clearwater, FL",501 to 1000 employees,2010,Company - Private,Business Services,$100 to $500 million (USD)
3,3,Data Scientist,$56K-$97K (Glassdoor est.),3.8,PNNL\n3.8,"Richland, WA",1001 to 5000 employees,1965,Government,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD)
4,4,Data Scientist,$86K-$143K (Glassdoor est.),2.9,Affinity Solutions\n2.9,"New York, NY",51 to 200 employees,1998,Company - Private,Business Services,Unknown / Non-Applicable


In [6]:
#Cleaning and organizing existing data in file1.
#Only include columns we want and rename them accordingly.
df3 = df[["Job_Title", "Salary_Estimate", "Company_Name", "Location", "Sector", "Type_of_ownership", "Size", "Rating"]]


In [10]:
df4 = df3.rename(columns = {
    "Job_Title": "Job Title",
    "Salary_Estimate": "Salary Range",
    "Company_Name": "Company Name",
    "Location": "Job Location",
    "Sector": "Sector",
    "Type_of_ownership": "Type of Ownership",
    "Size": "Size of Company",
    "Rating": "Rating of Company"
})

In [11]:
#Remove rows with per hour rate reported.
df5 = df4[df4["Salary Range"].str.contains("Per Hour")==False]

#Delete/replace extra text elements.
df6 = df5.replace("(Glassdoor est.)", "", regex = True)
df7 = df6.replace("(Employer est.)", "", regex = True)
df8 = df7.replace("Employer Provided Salary:", "", regex = True)
df8["Salary Range"] = df8["Salary Range"].replace("K", "000", regex = True)

#Remove rows with rating = -1.
df9 = df8[df8["Rating of Company"] != -1]

#Clean "Company Name" column.
df9[["Company Name", "Rating of Company"]] = df9["Company Name"].str.split("\n", expand=True)

#Split "Salary Range" into separate columns of "Low" and "High", cast as int.
df9[["Salary Range", "n"]] = df9["Salary Range"].str.split(" ", expand=True)
df10 = df9.drop(columns = "n")
df10[["Low End of Salary Range", "High End of Salary Range"]] = df10["Salary Range"].str.split("-", expand=True)
df10[["$", "Low End of Salary Range"]] = df10["Low End of Salary Range"].str.split("$", expand=True)
df10[["$$", "High End of Salary Range"]] = df10["High End of Salary Range"].str.split("$", expand=True)
df11 = df10.drop(columns = "$")
df12 = df11.drop(columns = "$$")
df12["Low End of Salary Range"] = df12["Low End of Salary Range"].astype(int)
df12[["High End of Salary Range", "n"]] = df12["High End of Salary Range"].str.split("(", expand=True)
df13 = df12.drop(columns = "n")
df13["High End of Salary Range"] = df13["High End of Salary Range"].astype(int)

#Organize columns in preferred order.
df14 = df13[["Job Title", "Low End of Salary Range", "High End of Salary Range", "Company Name", "Job Location", "Sector", "Type of Ownership", "Size of Company", "Rating of Company"]]

#Cast "Rating of Company" as float.
df14["Rating of Company"] = df14["Rating of Company"].astype(float)

C:\Users\sydne\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\sydne\AppData\Local\Temp/ipykernel_10016/4003718119.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df14["Rating of Company"] = df14["Rating of Company"].astype(float)


In [13]:
#Making "Middle of Salary Range" column.
a = df14["Low End of Salary Range"].tolist()
b = df14["High End of Salary Range"].tolist()
c = []
for i in range(len(a)):
    c.append((a[i]+b[i])/2)
df14["Middle of Salary Range"] = c
df15 = df14[["Job Title", "Low End of Salary Range", "Middle of Salary Range", "High End of Salary Range", "Company Name", "Job Location", "Sector", "Type of Ownership", "Size of Company", "Rating of Company"]]
df15["Middle of Salary Range"] = df15["Middle of Salary Range"].astype(int)

In [14]:
#Cleaning and organizing existing data in file2. Making "City, State" column out of "City" and "State".
df2["City, State"] = df2["City"] + ", " + df2["State"]
df16 = df2[["City, State", "Cost of Living Index"]]

In [15]:
#Create dictionary of keys=cities, values=COL. Loop through list of "Job Locations" and append COL number if "Job Locations" matches key in dictionary, otherwise append "NaN".
job_loc_list = df15["Job Location"].tolist()
city_state_list = df16["City, State"].tolist()
col_index_list = df16["Cost of Living Index"].tolist()
col_dict = {city_state_list[i]: col_index_list[i] for i in range(len(city_state_list))}
col_list = []
for i in range(len(job_loc_list)):
    if job_loc_list[i] in col_dict.keys():
        for key in col_dict:
            if job_loc_list[i] == key:
                col_list.append(col_dict[key])
    else:
        col_list.append("NaN")
df15["Cost of Living Index"] = col_list

In [16]:
#Calculate differential between middle of salary range and the COL for a given location.
#avg_col is average cost of living in United States as a single person ($38266 per year).
avg_sal = df15["Middle of Salary Range"].mean()
avg_col = 38266

In [17]:
#Create column for Salary index and generate values identically to how COL index was.
avg_sal_list = df15["Middle of Salary Range"].tolist()
avg_sal_index_list = []
for i in range(len(avg_sal_list)):
    avg_sal_index_list.append(round((avg_sal_list[i]/avg_sal)*100, 1))
df15["Average Salary Index"] = avg_sal_index_list

In [18]:
#Calculate differential, otherwise "NaN".
x = df15["Cost of Living Index"].tolist()
y = df15["Average Salary Index"].tolist()
z = []
for i in range(len(x)):
    if type(x[i]) == float:
        z.append(round((y[i]*avg_sal - x[i]*avg_col)/100))
    else:
        z.append("NaN")
df15["Salary and COL Differential"] = z
df15

,Job Title,Low End of Salary Range,Middle of Salary Range,High End of Salary Range,Company Name,Job Location,Sector,Type of Ownership,Size of Company,Rating of Company,Cost of Living Index,Average Salary Index,Salary and COL Differential
0,Data Scientist,53000,72000,91000,Tecolote Research,"Albuquerque, NM",Aerospace & Defense,Company - Private,501 to 1000 employees,3.8,92.9,70.1,36402
1,Healthcare Data Scientist,63000,87500,112000,University of Maryland Medical System,"Linthicum, MD",Health Care,Other Organization,10000+ employees,3.4,NaN,85.2,NaN
2,Data Scientist,80000,85000,90000,KnowBe4,"Clearwater, FL",Business Services,Company - Private,501 to 1000 employees,4.8,NaN,82.8,NaN
3,Data Scientist,56000,76500,97000,PNNL,"Richland, WA","Oil, Gas, Energy & Utilities",Government,1001 to 5000 employees,3.8,NaN,74.5,NaN
4,Data Scientist,86000,114500,143000,Affinity Solutions,"New York, NY",Business Services,Company - Private,51 to 200 employees,2.9,128.0,111.6,65566
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,"Sr Scientist, Immuno-Oncology - Oncology",58000,84500,111000,GSK,"Cambridge, MA",Biotech & Pharmaceuticals,Company - Public,10000+ employees,3.9,NaN,82.3,NaN
738,Senior Data Engineer,72000,102500,133000,Eventbrite,"Nashville, TN",Information Technology,Company - Public,1001 to 5000 employees,4.4,100.1,99.9,64233
739,"Project Scientist - Auton Lab, Robotics Institute",56000,73500,91000,Software Engineering Institute,"Pittsburgh, PA",Education,College / University,501 to 1000 employees,2.6,93.1,71.6,37865
740,Data Science Manager,95000,127500,160000,"Numeric, LLC","Allentown, PA",Business Services,Company - Private,1 to 50 employees,3.2,98.9,124.2,89634


In [19]:
df16 = df15.rename(columns = {
    "Job Title": "job_title",
    "Low End of Salary Range": "low_end_salary",
    "Middle of Salary Range": "middle_range_salary",
    "High End of Salary Range": "high_end_salary",
    "Company Name": "company_name",
    "Job Location": "job_location",
    "Sector": "sector",
    "Type of Ownership": "type_of_ownership",
    "Size of Company": "size_of_company",
    "Rating of Company": "rating_of_company",
    "Cost of Living Index": "cost_of_living_index",
    "Average Salary Index": "avg_salary_index",
    "Salary and COL Differential": "salary_col_differential",
})
df16.head()

,job_title,low_end_salary,middle_range_salary,high_end_salary,company_name,job_location,sector,type_of_ownership,size_of_company,rating_of_company,cost_of_living_index,avg_salary_index,salary_col_differential
0,Data Scientist,53000,72000,91000,Tecolote Research,"Albuquerque, NM",Aerospace & Defense,Company - Private,501 to 1000 employees,3.8,92.9,70.1,36402
1,Healthcare Data Scientist,63000,87500,112000,University of Maryland Medical System,"Linthicum, MD",Health Care,Other Organization,10000+ employees,3.4,NaN,85.2,NaN
2,Data Scientist,80000,85000,90000,KnowBe4,"Clearwater, FL",Business Services,Company - Private,501 to 1000 employees,4.8,NaN,82.8,NaN
3,Data Scientist,56000,76500,97000,PNNL,"Richland, WA","Oil, Gas, Energy & Utilities",Government,1001 to 5000 employees,3.8,NaN,74.5,NaN
4,Data Scientist,86000,114500,143000,Affinity Solutions,"New York, NY",Business Services,Company - Private,51 to 200 employees,2.9,128.0,111.6,65566


In [20]:
df16.to_csv('working_table.csv')